In [14]:
import sys
import numpy as np
import pandas as pd
import csv
import nltk
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file

# Everything has been split roughly into 80:20 training:testing from the datasets
####################################################################################################
# mammogram vectorization 
# 961 samples
# contains '?' values for undefined/missing
# label 0 for benign, 1 for malignant
df = pd.read_csv('datasets/mammograms/mammographic_masses.data')
mammograms = df.values

mammogram_features = mammograms[:,:-1]
mammogram_labels = mammograms[:,-1]

m_train, m_test, m_train_labels, m_test_labels = train_test_split(mammogram_features, mammogram_labels, test_size=.2, random_state=23)

####################################################################################################
# wisconsin breast cancer vectorization
# 669 samples
# contains '?' values for undefined/missing
# label 0 for benign, 1 for malignant

df1 = pd.read_csv('datasets/wisconsin_breast_cancer/breast-cancer-wisconsin.data')
wbc = df1.values

wbc_features = wbc[:,1:-1]
wbc_labels = wbc[:,-1]

wbc_labels /= 2
wbc_labels -= 1

wbc_train, wbc_test, wbc_train_labels, wbc_test_labels = train_test_split(wbc_features, wbc_labels, test_size=.2, random_state=1)


####################################################################################################
# banknote authentication set
# 1372 samples
# label 0 for real, 1 for forgery

df2 = pd.read_csv('datasets/banknote/data_banknote_authentication.txt')
bn = df2.values

bn_features = bn[:,:-1]
bn_labels = bn[:,-1]
bn_train, bn_test, bn_train_labels, bn_test_labels = train_test_split(bn_features, bn_labels, test_size=.2, random_state=10)

####################################################################################################
# covtype.binary
# 581,012 samples

cov_features, cov_labels = load_svmlight_file('datasets/covtype_binary/covtype.libsvm.binary')

cov_labels = cov_labels - 1

cov_train, cov_test, cov_train_labels, cov_test_labels = train_test_split(cov_features, cov_labels, test_size=.2, random_state=4)
print cov_train_labels

####################################################################################################
# spam_sms vectorization
# 

#df2 = pd.read_csv('datasets/spam_sms/SMSSpamCollection')

#sms_raw = pd.read_csv("datasets/spam_sms/SMSSpamCollection",)

####################################################################################################
# AUS rain prediction vectorization
# ACHTUNG: might need more preprocessing because of text values in some

#df1 = pd.read_csv("datasets/AUS_rain/weatherAUS.csv")
#df_features = df1.drop(['RISK_MM','RainTomorrow'], axis=1)
#df_labels = df1[['RainTomorrow']]

#aus_features = df_features.values
#aus_labels = df_labels.values

[0. 0. 1. ... 1. 1. 1.]
